# Análise de Candidatos ao Vestibular da UEA

-  Análise exploratória
    - [X] Verificar existência de dados nulos
    - [X] Formatar ``datanasc`` de str para datetime
    - [X] Formatar ``classificacao`` de float para int
    - [X] Criar atributo ``idade``
    - [X] Formatar atributo ``id_referencia_grupo``
    - [ ] Ordenar (na ordem que aparece na leitura do PDF) dataframe considerando: 
        1. Por ``ano``
        2. Por ``curso_nome_referencia``
        3. Por ``id_curso_referencia``
        4. Por ``classificacao`` 


## Pré-processamento dos dados

In [61]:
import numpy as np 
import pandas as pd

from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


### Carregando o dataset

In [62]:
data = pd.read_csv('../data/csv/candidates_info.csv', sep=',', encoding='utf-8')

In [63]:
data.head(5)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
0,6354130-0,EDIJANE PINHEIRO MARTINS,04/03/92,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019
1,6318761-2,JOEL DOS SANTOS DE SOUZA,09/03/97,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
2,6379123-4,CARINA DAS CHAGAS SEIXAS,23/10/96,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
3,6549853-4,IRLA DA SILVA PEREIRA,07/02/01,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019
4,6393350-0,IVANIR LIMA PEREIRA,11/03/91,317.0,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019


## Análise exploratória dos dados

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184389 entries, 0 to 184388
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   numinscricao           184389 non-null  object 
 1   nome                   184389 non-null  object 
 2   datanasc               184389 non-null  object 
 3   classificacao          184389 non-null  float64
 4   opcao_curso_2          184389 non-null  object 
 5   opcao_3_curso          184389 non-null  object 
 6   habilidades            184389 non-null  object 
 7   nota_final             184389 non-null  object 
 8   nota_cg                184389 non-null  object 
 9   nota_etapa_1           184389 non-null  object 
 10  disc1                  184389 non-null  int64  
 11  disc2                  184389 non-null  int64  
 12  disc3                  184389 non-null  int64  
 13  nota_ce                184389 non-null  object 
 14  nota_redacao           184389 non-nu

#### Verificando a existência de colunas com dados nulos

In [65]:
data.isnull().sum()

numinscricao             0
nome                     0
datanasc                 0
classificacao            0
opcao_curso_2            0
opcao_3_curso            0
habilidades              0
nota_final               0
nota_cg                  0
nota_etapa_1             0
disc1                    0
disc2                    0
disc3                    0
nota_ce                  0
nota_redacao             0
nota_etapa_2             0
opcao_curso_1            0
id_referencia_curso      0
nome_referencia_curso    0
ano                      0
dtype: int64

#### Formatando coluna 'datanasc' para o formato date

In [59]:
data['classificacao'] = data['classificacao'].astype(float)

In [60]:
data['classificacao']

0         31300.0
1         31400.0
2         31500.0
3         31600.0
4         31700.0
           ...   
184384     2400.0
184385     2500.0
184386     2600.0
184387      100.0
184388      200.0
Name: classificacao, Length: 184389, dtype: float64

## Criando novos atributos

In [8]:
def adjuste_year(year):
    if year <= 24:
        return 2000 + year
    else:
        return 1900 + year

def calculate_age(row):
    data_str = row['datanasc']
    current_year = row['ano']
    
    data = datetime.strptime(data_str, '%d/%m/%y')
    born_year = adjuste_year(data.year % 100)
    idade = current_year - born_year
    
    return idade

data['idade'] = data.apply(calculate_age, axis=1)

In [9]:
data[data['idade'] < 0]

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,...,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade


In [10]:
data.sample(2)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,...,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
7944,6562689-3,PEDRO HENRIQUE DOS SANTOS ERNESTO MACHADO,19/05/02,1,06067,07067,No,"58,002","63,095","63,095",...,5,3,"30,00","22,91","52,909",5067,6067,Medicina - Integral - Manaus,2019,17
164925,2407050-5,ESTER ALMEIDA CAUPER,27/02/05,73,03066,No,No,"51,240","53,571","53,571",...,5,7,"26,00","22,91","48,909",2066,3066,Sistemas de Informação - Integral - Manaus,2022,17


#### Padronizando as colunas referentes a opção de curso

In [11]:
data['opcao_curso_1'] = data['opcao_curso_1'].astype(str)
data['opcao_curso_1'] = 
data['id_referencia_curso'] = data['id_referencia_curso'].astype(str)

In [12]:
data['opcao_3_curso'] = data['opcao_3_curso'].apply(lambda x: x[1:] if str(x).startswith('0') else x)
data['opcao_curso_2'] = data['opcao_curso_2'].apply(lambda x: x[1:] if str(x).startswith('0') else x)

In [13]:
data.head(8)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,...,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
0,6354130-0,EDIJANE PINHEIRO MARTINS,04/03/92,313,No,No,No,"31,161","19,048","19,048",...,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019,27
1,6318761-2,JOEL DOS SANTOS DE SOUZA,09/03/97,314,No,No,No,"31,154","29,762","29,762",...,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,22
2,6379123-4,CARINA DAS CHAGAS SEIXAS,23/10/96,315,No,No,No,"31,154","29,762","29,762",...,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,23
3,6549853-4,IRLA DA SILVA PEREIRA,07/02/01,316,No,No,No,"31,113","30,952","30,952",...,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,18
4,6393350-0,IVANIR LIMA PEREIRA,11/03/91,317,No,No,No,"31,113","30,952","30,952",...,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,28
5,6612916-8,LAISA DOS SANTOS GALVAO,10/02/03,318,No,No,No,"31,046","25,000","25,000",...,2,3,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,16
6,6424859-3,ELCIA NEVES DOS SANTOS,26/12/01,319,No,No,No,"31,046","25,000","25,000",...,3,1,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,18
7,6386972-1,LAIZA DE FARIAS MAKLOUF,21/10/02,320,No,No,No,"31,004","26,190","26,190",...,3,2,"18,00","17,82","35,818",4069,4069,Odontologia - Integral - Manaus,2019,17


In [14]:
data.dtypes

numinscricao             object
nome                     object
datanasc                 object
classificacao             int32
opcao_curso_2            object
opcao_3_curso            object
habilidades              object
nota_final               object
nota_cg                  object
nota_etapa_1             object
disc1                     int64
disc2                     int64
disc3                     int64
nota_ce                  object
nota_redacao             object
nota_etapa_2             object
opcao_curso_1            object
id_referencia_curso      object
nome_referencia_curso    object
ano                       int64
idade                     int64
dtype: object

In [41]:
year_dataframes = {}
for year in data["ano"].unique():
    df_year = data.loc[data["ano"] == year]
    year_dataframes[year] = df_year

In [42]:
ordered_classifications = {}

for year, df_year in year_dataframes.items():
    year_info = {}
    for course_id in df_year['id_referencia_curso'].unique():
        df_course_year = df_year.loc[df_year['id_referencia_curso'] == course_id]
        df_course_year = df_course_year.sort_values("classificacao")
        year_info[course_id] = df_course_year
    ordered_classifications[year] = year_info

In [44]:
del year_info

In [45]:
dataframes_year = []

for ano, id_group in ordered_classifications.items():
    for id in id_group:
      df_year = pd.DataFrame.from_dict(id_group[id])
      dataframes_year.append(df_year)
full_df = pd.concat(dataframes_year, ignore_index=True)
# full_df.to_csv("../data/csv/lista_classificacao_vestibular_completo.csv", index=False)


In [46]:
del id, id_group, df_year

In [47]:
dados_vaga = pd.read_csv('../data/csv/approved_candidates.csv', sep=',', encoding='utf-8')

In [48]:
full_df['aprovado'] = full_df['nome'].isin(dados_vaga['nome'].values).astype(int)

In [49]:
years_full_df = full_df['ano'].unique()
years_dados_vaga = dados_vaga['ano'].unique()

for year in years_full_df:
    if year in years_dados_vaga:
        full_df_year = full_df[full_df['ano'] == year]
        dados_vaga_year = dados_vaga[dados_vaga['ano'] == year]

        full_df.loc[full_df_year.index, 'aprovado'] = full_df_year['nome'].isin(dados_vaga_year['nome'].values).astype(int)
        full_df['aprovado'] = full_df['aprovado'].astype(int)

In [50]:
def prefix(row):
  prefixo = row[0]
  return 1 if prefixo in ['1', '5'] else 0

full_df['egresso_publica'] = full_df['opcao_curso_1'].apply(prefix)

## Formatando Dataset para treino

In [51]:
columns = ['nota_final', 'nota_ce', 'nota_cg', 'nota_etapa_1', 'nota_etapa_2', 'nota_redacao']

for column in columns:
    full_df[column] = full_df[column].str.replace(',', '.').astype(float)


In [52]:
full_df.sample(2)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,...,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade,aprovado,egresso_publica
54522,6443807-4,ANA BEATRIZ CASTRO DE OLIVEIRA,29/06/02,184,2060,3060,No,34.344,32.143,32.143,...,20.0,16.55,36.545,1060,1060,Licenciatura em Pedagogia - Matutino - Manaus,2019,17,0,1
16097,6363640-9,DHULIA JACKELINE FERNANDES DA SILVA SALES,23/04/01,3,6067,7067,No,40.699,29.762,29.762,...,30.0,21.64,51.636,5067,7067,Medicina - Integral - Manaus,2019,18,0,1


In [53]:
full_df

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,...,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade,aprovado,egresso_publica
0,6659880-0,JOYCE MARIA GUIMARAES DE SOUZA,04/08/00,1,No,No,No,74.201,67.857,67.857,...,64.0,16.55,80.545,4069,4069,Odontologia - Integral - Manaus,2019,19,1,0
1,6660210-6,BRAYAN SANTOS DA SILVA,22/01/95,2,No,No,No,71.780,64.286,64.286,...,64.0,15.27,79.273,4069,4069,Odontologia - Integral - Manaus,2019,24,1,0
2,6812803-7,BRUNA HEGEIA ALBUQUERQUE DE SOUZA,28/08/92,3,No,No,No,69.589,61.905,61.905,...,62.0,15.27,77.273,4069,4069,Odontologia - Integral - Manaus,2019,27,1,0
3,6691348-9,SORAYA FASSIHA AMED MARTINS,25/01/02,4,No,No,No,69.217,59.524,59.524,...,56.0,22.91,78.909,4069,4069,Odontologia - Integral - Manaus,2019,17,1,0
4,6702990-6,ADRIANE VITORIA BATISTA DA SILVA,06/12/01,5,No,No,No,68.814,73.810,73.810,...,46.0,17.82,63.818,4069,4069,Odontologia - Integral - Manaus,2019,18,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184384,2368647-2,VERA LUCIA TRINDADE DO PRADO,25/04/94,24,No,No,No,26.078,21.429,21.429,...,18.0,12.73,30.727,8091,8091,Direito - Noturno - Manicoré,2022,28,0,0
184385,2396762-5,WASLEY SALOMAO DA SILVA NEVES,05/05/04,25,No,No,No,23.128,22.619,22.619,...,16.0,7.64,23.636,8091,8091,Direito - Noturno - Manicoré,2022,18,0,0
184386,2499585-1,LEANDRA DOS SANTOS LINHARES,10/12/83,26,No,No,No,20.994,10.714,10.714,...,16.0,15.27,31.273,8091,8091,Direito - Noturno - Manicoré,2022,39,0,0
184387,2405991-9,ICARO MARCOS PAVAO NEVES,20/10/02,1,No,No,No,50.230,38.095,38.095,...,42.0,20.36,62.364,9091,9091,Direito - Noturno - Manicoré,2022,20,0,0


In [57]:
full_df.drop(columns=['nome', 'disc1', 'disc2', 'disc3', 'habilidades', 'datanasc'], inplace=True)

In [58]:
full_df.to_csv('../data/dataset.csv', index=False)